In [ ]:
from dotenv import load_dotenv
load_dotenv()

from prefect.client import get_client
from prefect.states import Scheduled
from prefect.client.schemas.filters import FlowRunFilter, DeploymentFilter, FlowRunFilterStateName

DEPLOYMENT_ID = "1f41ea3b-415a-4daf-ae14-4c838c14675d"
PAGE_SIZE = 200
from datetime import datetime, timedelta, timezone
from prefect.client import get_client
from prefect.states import Scheduled
from prefect.client.schemas.filters import (
    FlowRunFilter,
    FlowRunFilterStateName,
    FlowRunFilterExpectedStartTime,
    DeploymentFilter
)

DEPLOYMENT_ID = "1f41ea3b-415a-4daf-ae14-4c838c14675d"
PAGE_SIZE = 200

async def reschedule_recent_late_and_pending(deployment_id):
    offset = 0
    total_rescheduled = 0
    one_week_ago = datetime.now(timezone.utc) - timedelta(days=7)

    async with get_client() as client:
        while True:
            flow_run_filter = FlowRunFilter(
                state=FlowRunFilterStateName(any_=["Late", "Pending"]),
                expected_start_time=FlowRunFilterExpectedStartTime(after_=one_week_ago)
            )
            deployment_filter = DeploymentFilter(id={"any_": [deployment_id]})

            runs = await client.read_flow_runs(
                flow_run_filter=flow_run_filter,
                deployment_filter=deployment_filter,
                limit=PAGE_SIZE,
                offset=offset
            )

            if not runs:
                break

            for run in runs:
                state_name = run.state.name.lower()
                if state_name in {"late", "pending"}:
                    print(f"Force-scheduling run: {run.id} ({state_name})")
                    await client.set_flow_run_state(run.id, Scheduled(), force=True)
                    total_rescheduled += 1
                else:
                    print(f"Skipping run: {run.id} ({run.state.name})")

            offset += PAGE_SIZE

    print(f"\n✅ Force-scheduled {total_rescheduled} Late/Pending runs from the past 7 days.")


await reschedule_recent_late_and_pending(DEPLOYMENT_ID)